# 

In [3]:
import pandas as pd

dataset = pd.read_csv('dataset.csv')

dataset.head()

,id,matchid,teamid,player,championid,position,kills,deaths,assists,killingsprees,...,win,goldearned,champlvl,totheal,totdmgtaken,towerkills,inhibkills_y,baronkills,dragonkills,harrykills
0,10,10,100,2,267,SUPP,0,2,12,0,...,0,9496,14,11707,17769,5,0,0,0,0
1,11,10,100,3,119,ADC,7,8,5,1,...,0,13136,14,2283,25627,5,0,0,0,0
2,17,10,200,9,222,ADC,15,3,9,2,...,1,15970,16,2802,17655,10,3,1,3,1
3,18,10,200,10,161,SUPP,4,5,19,1,...,1,12978,16,3242,13443,10,3,1,3,1
4,19,11,100,1,115,SUPP,2,7,5,0,...,0,7792,12,104,11576,2,0,0,0,0


In [8]:
# Add a KDA column where KDA = (kills + assists) / deaths
# Handle cases where deaths are 0 to avoid division by zero
dataset['KDA'] = (dataset['kills'] + dataset['assists']) / dataset['deaths'].replace(0, 1)

# Drop the kills, deaths, and assists columns
dataset = dataset.drop(columns=['kills', 'deaths', 'assists'])

# Save the updated dataset to a CSV file for inspection
dataset.to_csv("updated_dataset_with_kda.csv", index=False)

# Display the first few rows of the updated dataset
print(dataset.head())


   id  matchid  teamid  player  championid position  killingsprees  \
0  10       10     100       2         267     SUPP              0   
1  11       10     100       3         119      ADC              1   
2  17       10     200       9         222      ADC              2   
3  18       10     200      10         161     SUPP              1   
4  19       11     100       1         115     SUPP              0   

   totdmgdealt  visionscore  win  goldearned  champlvl  totheal  totdmgtaken  \
0        25995           30    0        9496        14    11707        17769   
1       171568           26    0       13136        14     2283        25627   
2       182680           12    1       15970        16     2802        17655   
3        85785           71    1       12978        16     3242        13443   
4        46790           26    0        7792        12      104        11576   

   towerkills  inhibkills_y  baronkills  dragonkills  harrykills  KDA  
0           5             

In [10]:
from sklearn.preprocessing import MinMaxScaler

# Define ID-related columns that should not be normalized
id_columns = ['id', 'matchid', 'teamid', 'player', 'championid']  # Adjust this list based on your dataset

# Select numeric columns excluding ID-related columns
numeric_columns_to_normalize = dataset.select_dtypes(include=['int64', 'float64']).columns.difference(id_columns)

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Apply normalization only to non-ID numeric columns
dataset[numeric_columns_to_normalize] = scaler.fit_transform(dataset[numeric_columns_to_normalize])

# Save the updated dataset to a CSV file for inspection
dataset.to_csv("normalized_dataset_with_kda_excluding_ids.csv", index=False)

# Display the first few rows of the updated dataset
print(dataset.head())


   id  matchid  teamid  player  championid position  killingsprees  \
0  10       10     100       2         267     SUPP       0.000000   
1  11       10     100       3         119      ADC       0.090909   
2  17       10     200       9         222      ADC       0.181818   
3  18       10     200      10         161     SUPP       0.090909   
4  19       11     100       1         115     SUPP       0.000000   

   totdmgdealt  visionscore  win  goldearned  champlvl   totheal  totdmgtaken  \
0     0.999612     0.167598  0.0    0.216474  0.764706  0.118057     0.152240   
1     0.999680     0.145251  0.0    0.322498  0.764706  0.023022     0.219565   
2     0.999685     0.067039  1.0    0.405045  0.882353  0.028256     0.151263   
3     0.999640     0.396648  1.0    0.317896  0.882353  0.032693     0.115176   
4     0.999622     0.145251  0.0    0.166841  0.647059  0.001049     0.099180   

   towerkills  inhibkills_y  baronkills  dragonkills  harrykills       KDA  
0      0.3125  

In [16]:
dataset.dtypes

id                 int64
matchid            int64
teamid             int64
player             int64
championid         int64
position          object
killingsprees    float64
totdmgdealt      float64
visionscore      float64
win              float64
goldearned       float64
champlvl         float64
totheal          float64
totdmgtaken      float64
towerkills       float64
inhibkills_y     float64
baronkills       float64
dragonkills      float64
harrykills       float64
KDA              float64
dtype: object

In [20]:
!pip install ace_tools

In [28]:
from sklearn.decomposition import TruncatedSVD
import numpy as np

# Step 1: Create ADC-Support Pair Dataset
adc_rows = dataset[dataset['position'] == 'ADC']
support_rows = dataset[dataset['position'] == 'SUPP']

# Merge ADC rows with corresponding Support rows
adc_with_support = pd.merge(
    adc_rows,
    support_rows[['matchid', 'teamid', 'championid']],
    on=['matchid', 'teamid'],
    suffixes=('', '_support')
)

# Rename the support champion column
adc_with_support.rename(columns={'championid_support': 'support_champion'}, inplace=True)

# Drop unnecessary columns
adc_with_support = adc_with_support.drop(columns=['id', 'player', 'position'])

# Display the processed dataset
print(adc_with_support.head())

# Step 2: Prepare Data for SVD
# Create a matrix where rows are ADCs, columns are Supports, and values are win rates (or another metric)
matrix = adc_with_support.pivot_table(
    index='championid', 
    columns='support_champion', 
    values='win', 
    aggfunc='mean'
).fillna(0)

# Step 3: Perform SVD
svd = TruncatedSVD(n_components=10, random_state=42)
latent_factors = svd.fit_transform(matrix)

# Step 4: Make Recommendations
# Compute the predicted values by reconstructing the matrix
predicted_matrix = np.dot(latent_factors, svd.components_)

# Convert predictions back to DataFrame
predicted_df = pd.DataFrame(predicted_matrix, index=matrix.index, columns=matrix.columns)

# Step 5: Recommend Top Support Champions for Each ADC
def recommend_supports(adc_id, top_n=5):
    if adc_id not in predicted_df.index:
        return f"ADC {adc_id} not found in the dataset."
    recommendations = predicted_df.loc[adc_id].sort_values(ascending=False).head(top_n)
    return recommendations

# Example: Get top 5 recommended supports for a specific ADC
example_adc_id = adc_with_support['championid'].iloc[0]  # Replace with your ADC ID
recommendations = recommend_supports(example_adc_id)
print("champion id: ",example_adc_id)
# Display the recommendations
recommendations


   matchid  teamid  championid  killingsprees  totdmgdealt  visionscore  win  \
0       10     100         119       0.090909     0.999680     0.145251  0.0   
1       10     200         222       0.181818     0.999685     0.067039  1.0   
2       11     100          69       0.000000     0.999629     0.089385  0.0   
3       11     200          51       0.272727     0.999676     0.100559  1.0   
4       12     100         119       0.090909     0.999635     0.022346  0.0   

   goldearned  champlvl   totheal  totdmgtaken  towerkills  inhibkills_y  \
0    0.322498  0.764706  0.023022     0.219565      0.3125      0.000000   
1    0.405045  0.882353  0.028256     0.151263      0.6250      0.230769   
2    0.188745  0.647059  0.030283     0.141976      0.1250      0.000000   
3    0.347052  0.823529  0.035789     0.165580      0.6250      0.230769   
4    0.194192  0.588235  0.022972     0.154493      0.0625      0.000000   

   baronkills  dragonkills  harrykills       KDA  support_cham

support_champion
60     0.942070
432    0.892673
69     0.886181
75     0.835467
59     0.832444
Name: 119, dtype: float64

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Step 1: Create ADC-Support Pair Dataset
adc_rows = dataset[dataset['position'] == 'ADC']
support_rows = dataset[dataset['position'] == 'SUPP']

# Merge ADC rows with corresponding Support rows
adc_with_support = pd.merge(
    adc_rows,
    support_rows[['matchid', 'teamid', 'championid']],
    on=['matchid', 'teamid'],
    suffixes=('', '_support')
)

# Rename the support champion column
adc_with_support.rename(columns={'championid_support': 'support_champion'}, inplace=True)

# Drop unnecessary columns
adc_with_support = adc_with_support.drop(columns=['id', 'player', 'position'])

# Step 2: Feature Selection
# Define features (X) and target (y)
X = adc_with_support.drop(columns=['support_champion', 'matchid', 'teamid'])
y = adc_with_support['support_champion']

# Step 3: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Train the Model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Step 5: Evaluate the Model
y_pred = model.predict(X_test)

# Step 6: Recommend Supports
def recommend_supports_for_adc(adc_features, model, top_n=5):
    probabilities = model.predict_proba([adc_features])[0]
    support_champion_indices = probabilities.argsort()[-top_n:][::-1]
    support_champion_ids = model.classes_[support_champion_indices]
    recommendations = {support_id: probabilities[idx] for support_id, idx in zip(support_champion_ids, support_champion_indices)}
    return recommendations

# Example: Use the first row of X_test as input
example_adc_features = X_test.iloc[0]
recommendations = recommend_supports_for_adc(example_adc_features, model)
print("\nRecommended Supports with Probabilities:\n", recommendations)



Recommended Supports with Probabilities:
 {43: 0.18, 53: 0.11, 117: 0.09, 412: 0.09, 57: 0.08}


/Users/froguro/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [36]:
adc_with_support.head()

,matchid,teamid,championid,killingsprees,totdmgdealt,visionscore,win,goldearned,champlvl,totheal,totdmgtaken,towerkills,inhibkills_y,baronkills,dragonkills,harrykills,KDA,support_champion
0,10,100,119,0.090909,0.999680,0.145251,0.0,0.322498,0.764706,0.023022,0.219565,0.3125,0.000000,0.0,0.000000,0.0,0.031250,267
1,10,200,222,0.181818,0.999685,0.067039,1.0,0.405045,0.882353,0.028256,0.151263,0.6250,0.230769,0.2,0.428571,0.5,0.166667,161
2,11,100,69,0.000000,0.999629,0.089385,0.0,0.188745,0.647059,0.030283,0.141976,0.1250,0.000000,0.0,0.000000,0.0,0.020833,115
3,11,200,51,0.272727,0.999676,0.100559,1.0,0.347052,0.823529,0.035789,0.165580,0.6250,0.230769,0.0,0.285714,0.0,0.066667,43
4,12,100,119,0.090909,0.999635,0.022346,0.0,0.194192,0.588235,0.022972,0.154493,0.0625,0.000000,0.0,0.000000,0.0,0.023438,40


In [40]:
winrate_data_path = 'adc_support_duos_with_winrate.csv'
winrate_data = pd.read_csv(winrate_data_path)


# Step 4: Prepare Recommendations and Compare with Winrates
test_set_with_predictions = X_test.copy()
test_set_with_predictions['predicted_support'] = y_pred
test_set_with_predictions['adc_champion'] = X_test['championid']  # Assuming `championid` is the ADC column

# Update the merge to use the correct column names
test_set_with_predictions = test_set_with_predictions.merge(
    winrate_data[['ADC', 'Support', 'Winrate']],
    left_on=['adc_champion', 'predicted_support'],
    right_on=['ADC', 'Support'],
    how='left'
)

# Calculate the average win rate of the recommendations
average_winrate = test_set_with_predictions['Winrate'].mean()

# Output the result
print(f"Average Win Rate of Recommended Duos: {average_winrate:.2%}")


Average Win Rate of Recommended Duos: 51.01%


In [38]:
print(winrate_data.columns)


Index(['ADC', 'Support', 'Wins', 'Total Games', 'Winrate'], dtype='object')


In [42]:
X_test

,championid,killingsprees,totdmgdealt,visionscore,win,goldearned,champlvl,totheal,totdmgtaken,towerkills,inhibkills_y,baronkills,dragonkills,harrykills,KDA
278448,51,0.181818,0.999779,0.162011,0.0,0.533089,1.000000,0.038099,0.144358,0.2500,0.000000,0.2,0.000000,0.0,0.180556
24415,236,0.272727,0.999690,0.150838,0.0,0.448619,0.941176,0.041739,0.242090,0.4375,0.076923,0.2,0.428571,0.0,0.077381
229884,202,0.272727,0.999661,0.000000,0.0,0.299342,0.764706,0.025241,0.135773,0.3125,0.076923,0.0,0.000000,0.0,0.034722
174844,119,0.363636,0.999711,0.000000,1.0,0.512233,0.941176,0.039833,0.234456,0.6875,0.230769,0.4,0.285714,0.5,0.120833
227501,498,0.181818,0.999649,0.000000,0.0,0.217669,0.647059,0.023214,0.209173,0.0625,0.000000,0.0,0.000000,0.5,0.017361
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169297,236,0.000000,0.999649,0.134078,1.0,0.224455,0.705882,0.021994,0.097844,0.5625,0.153846,0.0,0.285714,0.5,0.125000
71333,81,0.181818,0.999654,0.067039,0.0,0.262204,0.705882,0.035245,0.160662,0.1250,0.000000,0.0,0.000000,0.0,0.050000
16161,67,0.363636,0.999734,0.000000,1.0,0.530700,1.000000,0.096194,0.363375,0.6250,0.307692,0.4,0.428571,0.0,0.056250
298198,67,0.090909,0.999631,0.000000,0.0,0.173628,0.647059,0.025745,0.188447,0.0000,0.000000,0.0,0.000000,0.0,0.011574
